<a href="https://colab.research.google.com/github/RodolfoFerro/satelitesyneuronas/blob/main/notebooks/Redes_neuronales_convolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neuronales convolucionales


### **Convoluciones en imágenes**

Exploremos qué sucede cuando barremos un filtro (kernel) sobre una imagen utilizando una convolución.

**Spoiler:** Intentemos escalar posibles resultados al tener muchos filtros dentro de una red neuronal.

In [ ]:
import numpy as np
from scipy import datasets
import matplotlib.pyplot as plt


# We load a sample image
img = datasets.ascent()

plt.imshow(img, cmap='gray')
plt.grid(False)
plt.axis('off')
plt.show()

Creamos una copia de la imagen.

In [ ]:
img_transformed = np.copy(img)
size_x = img_transformed.shape[0]
size_y = img_transformed.shape[1]

Definimos un filtro a utilizar.

In [ ]:
# Let's experiment with different values

filter = [[1, 2, 1], [2, 4, 2], [1, 2, 1]]
# filter = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
# filter = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]

weight = 1 / 8

Realizamos las operaciones.

In [ ]:
for x in range(1, size_x - 1):
  for y in range(1, size_y - 1):
      convolution = 0.0
      convolution = convolution + (img[x - 1, y - 1] * filter[0][0])
      convolution = convolution + (img[x, y - 1] * filter[0][1])
      convolution = convolution + (img[x + 1, y - 1] * filter[0][2])
      convolution = convolution + (img[x - 1, y] * filter[1][0])
      convolution = convolution + (img[x, y] * filter[1][1])
      convolution = convolution + (img[x + 1, y] * filter[1][2])
      convolution = convolution + (img[x - 1, y + 1] * filter[2][0])
      convolution = convolution + (img[x, y + 1] * filter[2][1])
      convolution = convolution + (img[x + 1, y + 1] * filter[2][2])
      convolution = convolution * weight

      if convolution < 0:
        convolution = 0
      if convolution > 255:
        convolution = 255

      img_transformed[x, y] = convolution

Veamos los resultados de convolución.

In [ ]:
plt.imshow(img_transformed, cmap='gray')
plt.grid(False)
plt.axis('off')
plt.show()


### **Pooling en imágenes**

Exploremos qué sucede cuando reducimos la información de una imagen a través de pooling.


In [ ]:
import numpy as np
import skimage.measure


img_transformed = np.copy(img)

plt.imshow(img_transformed, cmap='gray')
plt.grid(False)
plt.axis('off')
plt.show()

In [ ]:
img.shape

In [ ]:
img_transformed = skimage.measure.block_reduce(img_transformed, (2,2), np.max)

plt.imshow(img_transformed, cmap='gray')
plt.grid(False)
plt.axis('off')
plt.show()

In [ ]:
img_transformed.shape


### **Redes neuronales convolucionales**

**Spoiler:** Nuevamente, intentemos escalar posibles resultados al tener muchos filtros dentro de una red neuronal.

Para ello, crearemos un modelo de red neuronal convolucional profunda, que utilice, precisamente, convoluciones en sus capas.

Nos basaremos en un modelo LeNet5 propuesto por un gran investigador, Yann LeCun:

<center>
    <img src="https://www.datasciencecentral.com/wp-content/uploads/2021/10/1lvvWF48t7cyRWqct13eU0w.jpeg" width="60%">
</center>

#### El dataset a utilizar: Trees in Satellite Imagery

El dataset está compuesto por imágenes de 64x64 pixeles, que contienen un conjunto de 2 categorías.

El dataset fue descargado de: https://www.kaggle.com/datasets/mcagriaksoy/trees-in-satellite-imagery


In [ ]:
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/refs/heads/main/assets/data/trees.zip
!unzip trees
!mv Trees\ in\ Satellite\ Imagery trees_dataset

In [ ]:
!ls

In [ ]:
import os
import cv2
import numpy as np

dataset_path = 'trees_dataset'
classes = ['Trees', 'NoTrees']
img_size = 64
images = []
labels = []

for label, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

print(f"Loaded {len(images)} images with shape {images.shape}")
print(f"Loaded {len(labels)} labels with shape {labels.shape}")

In [ ]:
from sklearn.model_selection import train_test_split

training_images, testing_images, training_labels, testing_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training images shape: {training_images.shape}")
print(f"Testing images shape: {testing_images.shape}")
print(f"Training labels shape: {training_labels.shape}")
print(f"Testing labels shape: {testing_labels.shape}")

¿Cómo se ven estos valores? Despleguemos una imagen de entrenamiento y una etiqueta de entrenamiento para saber.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=200)


# Set index of image to be seen
img_index = 0

# Plot image
plt.imshow(training_images[img_index], cmap='gray')
plt.axis(False)

print("Label:", training_labels[img_index])
print("Matrix:", training_images[img_index])

Notarás que todos los valores están entre 0 y 255. Si estamos entrenando una red neuronal, por varias razones es más fácil si transformamos los valores para tratar todos con valores entre 0 y 1.

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

Notemos que cada imagen es sólo una matriz de 28x28 pixeles, sólo con 1 canal de color.

In [ ]:
training_images[0].shape

In [ ]:
cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Input((64, 64, 3)),

    # First conv layer + subsampling
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Second conv layer + subsampling
    # TODO. Conv2D -> 256, (3, 3), ReLU
    # TODO. MaxPool

    # Third layer (flatten)
    tf.keras.layers.Flatten(),

    # Fourth layer (dense)
    # TODO. Dense -> 128, ReLU

    # Fifth layer (output)
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
cnn_model.compile(
    optimizer=tf.optimizers.SGD(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
cnn_model.fit(training_images, training_labels, epochs=2)

In [ ]:
cnn_model.evaluate(test_images, test_labels)

In [ ]:
import random

test_index = random.randint(0, 2080 - 1)

plt.imshow(test_images[test_index], cmap='gray')
plt.axis(False)

print("Label:", test_labels[test_index])
input_image = np.reshape(test_images[test_index], (1, 28, 28, 3))
prediction = cnn_model.predict(input_image)
print("Prediction:", np.argmax(prediction))

**Reto:** ¿Puedes mejorar aún más el modelo?

Te recomiendo explorar lo siguiente:
- Modifica el número de capas y parámetros de convolución por capa
- Modifica el número de épocas de entrenamiento
- Explora resultados con otros conjuntos de datos
- ¿Exportar modelos entrenados? Ojo: https://www.tensorflow.org/guide/keras/save_and_serialize?hl=es-419

> **Para resolver la tarea, el reto es:** Mejor accuracy obtenido en la clase.

**Puedes explorar:**
- El número de capas.
- Las épocas de entrenamiento.
- Las funciones de activación.
- Investigar otras capas.

---
> Contenido creado por **Rodolfo Ferro**. Contacto: [ferro@cimat.mx](ferro@cimat.mx) <br>
[**Clubes de Ciencia México**](https://clubesdeciencia.mx/), 2025.